In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,1072.01,0.00,0.02,0.0,0.00,-0.00,0.00
3,0.00,328.28,-0.00,-0.0,-0.00,0.00,0.00
4,0.02,-0.00,82.07,-0.0,-0.00,-0.00,-0.00
5,0.00,-0.00,-0.00,15.7,-0.00,0.00,-0.00
6,0.00,-0.00,-0.00,-0.0,2.74,0.00,0.00
7,-0.00,0.00,-0.00,0.0,0.00,0.45,0.00
8,0.00,0.00,-0.00,-0.0,0.00,0.00,0.04


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00006,0.00001,0.00005,-0.00005,0.00006
3,0.00000,1.00000,-0.00002,-0.00005,-0.00004,0.00001,0.00001
4,0.00006,-0.00002,1.00000,-0.00003,-0.00003,-0.00004,-0.00001
5,0.00001,-0.00005,-0.00003,1.00000,-0.00002,0.00001,-0.00001
6,0.00005,-0.00004,-0.00003,-0.00002,1.00000,0.00002,0.00001
7,-0.00005,0.00001,-0.00004,0.00001,0.00002,1.00000,0.00005
8,0.00006,0.00001,-0.00001,-0.00001,0.00001,0.00005,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.02827588488392993

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.64667562e+05 1.49673438e+04 8.97740890e+02 3.08640345e+01
 8.37050236e-01 1.71796214e-02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998350,-0.055981,-0.012535,-0.002384,-0.000413,-0.000070,-0.000006
1,0.055203,0.996971,-0.053907,-0.009578,-0.001667,-0.000275,-0.000027
2,0.015346,0.052574,0.997205,-0.050111,-0.008356,-0.001383,-0.000135
3,0.003637,0.011916,0.048893,0.996866,-0.060167,-0.009564,-0.000931
4,0.000838,0.002752,0.011004,0.058630,0.994566,-0.084904,-0.007931
5,0.000210,0.000684,0.002737,0.014288,0.082997,0.991622,-0.097903
6,0.000041,0.000134,0.000536,0.002798,0.016034,0.096869,0.995164


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00164977, 0.00302855, 0.00279513, 0.00313394, 0.00543397,
       0.00837796, 0.00483612])